# Features <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/feature_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook showcases some of the various features of `yfiles-jupyter-graphs-for-neo4j`.

For a detailed description for the different mappings, check out the widget [documentation](https://yworks.github.io/yfiles-jupyter-graphs/)


In [1]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.9 MB/s eta 0:00:00


You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [2]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/feature_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the database

In [3]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com"
NEO4J_USERNAME = "fincen"
NEO4J_PASSWORD = "fincen"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = "fincen")

g = Neo4jGraphWidget(driver)

## Use heat mapping

In [4]:
max_amount = 120000000
min_amount = 50000

def heat_mapping(element):
    if "amount" in element["properties"]:
        amount = element["properties"]["amount"]
        normalized_value = (amount - min_amount) / (max_amount - min_amount)
        transformed_value = -1 * (1 - normalized_value) ** 2 + 1
        return max(0, min(1, transformed_value))

g.add_node_configuration("Filing", heat=heat_mapping)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

GraphWidget(layout=Layout(height='690px', width='100%'))

## Visualize geospatial data

In [5]:
def node_coordinate_mapping(node):
    return (node["properties"]["location"]["y"], node["properties"]["location"]["x"]) if "location" in node["properties"].keys() else None

def filing_coordinate_mapping(node):
    return (node["properties"]["beneficiary_lat"], (node["properties"]["beneficiary_lng"])) if "beneficiary_lat" in node["properties"].keys() else None

g.add_node_configuration(["Country", "Entity"], coordinate=node_coordinate_mapping)
g.add_node_configuration("Filing", coordinate=filing_coordinate_mapping)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 50", layout="map")

GraphWidget(layout=Layout(height='800px', width='100%'))

## Configure item visualization

In [6]:

g.add_relationship_configuration("CONCERNS", thickness_factor= 0.5)
g.add_relationship_configuration("FILED", thickness_factor= 2)
g.del_node_configuration("Entity")
g.add_node_configuration("Filing", size= lambda node:  (55 * (1 + heat_mapping(node)), 55 * (1 + heat_mapping(node))))

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

GraphWidget(layout=Layout(height='690px', width='100%'))

## Configure grouping

The widget supports various grouping options, see [grouping.ipynb](./grouping.ipynb) for more details.

In [7]:
# replaces "COUNTRY" relationships with a grouped hierarchy
g.add_parent_relationship_configuration("COUNTRY")

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

GraphWidget(layout=Layout(height='690px', width='100%'))

## Configure node-to-cell mapping

The node-to-cell mapping allows to fine-tune layout results by assigning preferred cell constraints for specific nodes.

This is particularly useful to highlight specific items structurally aside from visual features like color or size.

In [8]:
g.del_parent_relationship_configuration("COUNTRY")

# highlight Bank of New York and China Construction Bank
def get_cell_mapping(node):
    name = node["properties"].get("name")
    if name == "The Bank of New York Mellon Corp." or name == "China Construction Bank Corporation":
        return (0,0)
    return (1,0)

g.node_cell_mapping = get_cell_mapping

# layouts interpret node-to-cell mapping differently
g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25", layout='hierarchic')

GraphWidget(layout=Layout(height='690px', width='100%'))